In [1]:
from rdkit import Chem
from rdkit.Chem import AllChem, Draw
import numpy as np

import openmm as mm
import openmm.app as app
from openmm import unit
import pdbfixer
from openff.toolkit.topology import Molecule
from openmmforcefields.generators import GAFFTemplateGenerator, SMIRNOFFTemplateGenerator
import mdtraj as md

In [2]:
fixer = pdbfixer.PDBFixer("data/7zzs_zinc.pdb")

# remove heterogen
fixer.removeHeterogens()
fixer.findMissingResidues()
# 末端残基の削除
chains = list(fixer.topology.chains())
keys = fixer.missingResidues.keys()
for key in list(keys):
    chain = chains[key[0]]
    if key[1] == 0 or key[1] == len(list(chain.residues())):
        del fixer.missingResidues[key]
# 非標準残基、欠損原子の確認
fixer.findNonstandardResidues()
fixer.replaceNonstandardResidues()
fixer.findMissingAtoms()
fixer.addMissingAtoms()
# 水素原子の付与
ph = 7.0
fixer.addMissingHydrogens(ph)


In [3]:
# Znの抽出
fixer2 = pdbfixer.PDBFixer("data/7zzs_zinc.pdb")
mod_zn = app.Modeller(fixer2.topology,fixer2.positions)
toDelete = []
for v in fixer2.topology.residues():
    if v.name != "ZN":
        toDelete.append(v)
mod_zn.delete(toDelete)

In [4]:
rdkit_mol = Chem.MolFromPDBFile("data/vorinostat.pdb")
#rdkit_mol_split = Chem.rdmolops.SplitMolByPDBResidues(rdkit_mol)

# extract the ligand and remove any already present hydrogens
ligand = Chem.RemoveHs(rdkit_mol)

# assign bond orders from template
reference_mol = Chem.MolFromSmiles("O=C(Nc1ccccc1)CCCCCCC(=O)NO")
prepared_ligand = AllChem.AssignBondOrdersFromTemplate(reference_mol, ligand)
prepared_ligand.AddConformer(ligand.GetConformer(0))

# protonate ligand
prepared_ligand = Chem.rdmolops.AddHs(prepared_ligand, addCoords=True)
prepared_ligand = Chem.MolFromMolBlock(Chem.MolToMolBlock(prepared_ligand))

[16:30:59] WARNING: More than one matching pattern found - picking one



In [5]:
off_mol = Molecule.from_rdkit(prepared_ligand)

# add names for atoms
element_counter_dict = {}
for off_atom, rdkit_atom in zip(off_mol.atoms, rdkit_mol.GetAtoms()):
    element = rdkit_atom.GetSymbol()
    if element in element_counter_dict.keys():
        element_counter_dict[element] += 1
    else:
        element_counter_dict[element] = 1
    off_atom.name = element + str(element_counter_dict[element])

# convert from OpenFF to OpenMM
off_mol_topology = off_mol.to_topology()
mol_topology = off_mol_topology.to_openmm()
mol_positions = off_mol.conformers[0]

# convert units from Ångström to nanometers
# since OpenMM works in nm
mol_positions = mol_positions.to("nanometers")

# combine topology and positions in modeller object
omm_mol = app.Modeller(mol_topology, mol_positions)

In [6]:
md_protein_topology = md.Topology.from_openmm(fixer.topology)  # using mdtraj for protein top
md_ligand_topology = md.Topology.from_openmm(omm_mol.topology)  # using mdtraj for ligand top
md_ion_topology = md.Topology.from_openmm(mod_zn.topology)
md_complex_topology = md_protein_topology.join(md_ligand_topology)  # add them together
md_complex_topology = md_complex_topology.join(md_ion_topology)
complex_topology = md_complex_topology.to_openmm()

# combine positions
total_atoms = len(fixer.positions) + len(omm_mol.positions) + len(mod_zn.positions)

# create an array for storing all atom positions as tupels containing a value and a unit
# called OpenMM Quantities
complex_positions = unit.Quantity(np.zeros([total_atoms, 3]), unit=unit.nanometers)
complex_positions[: len(fixer.positions)] = fixer.positions  # add protein positions
complex_positions[len(fixer.positions) :-len(mod_zn.positions)] = omm_mol.positions  # add ligand positions
complex_positions[-len(mod_zn.positions):] = mod_zn.positions

/opt/miniconda3/envs/mdenv/lib/python3.11/site-packages/openmm/unit/quantity.py:753: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  self._value[key] = value / self.unit


In [7]:
FF = app.ForceField('amber/ff14SB.xml', 
                    'amber/tip3pfb_standard.xml')
smff = SMIRNOFFTemplateGenerator(
    molecules=off_mol
)
FF.registerTemplateGenerator(smff.generator)

In [8]:
complex_model = app.Modeller(complex_topology, complex_positions)
complex_model.addSolvent(FF,padding=1.2*unit.nanometers,ionicStrength=0.15*unit.molar)
top = complex_model.getTopology()
pos = complex_model.getPositions()
app.PDBFile.writeFile(top,pos,open("data/7zzs_SMIRNOFF_processed.pdb","w"))

---

In [9]:
import time

from openmm import *
from openmm.app import *
from openmm import unit

In [10]:
nonbondedMethod = PME
nonbondedCutoff = 1.0 * unit.nanometers
ewaldErrorTolerance = 5e-4
constraints = HBonds
rigidWater = True
constraintTolerance = 1e-6

temperature = 300 * unit.kelvin
friction = 1.0 / unit.picoseconds
pressure = 1.0 * unit.atmospheres
barostatInterval = 25

dt = 2.0 * unit.femtoseconds
equilibrationSteps = 50000
steps = 50000000

platform = Platform.getPlatformByName("CUDA")
platformProperties = {"Precision":"single"}

reporterStep = 10000
checkpointStep = 10000

print("equilibration time: {} ps".format(np.floor(dt*equilibrationSteps / unit.picoseconds)))
print("simulation time: {} ns".format(np.floor(dt*steps / unit.nanoseconds)))

equilibration time: 100.0 ps
simulation time: 100.0 ns


In [11]:
system = FF.createSystem(complex_model.topology,
                         nonbondedMethod=nonbondedMethod,
                         nonbondedCutoff=nonbondedCutoff,
                         constraints=constraints,
                         rigidWater=rigidWater,
                         ewaldErrorTolerance=ewaldErrorTolerance)
system.addForce(MonteCarloBarostat(pressure,temperature,barostatInterval))

# integrator
integrator = LangevinMiddleIntegrator(temperature,friction,dt)
integrator.setConstraintTolerance(constraintTolerance)

# simulation const
simulation = Simulation(complex_model.topology,system,integrator,platform,platformProperties)
simulation.context.setPositions(complex_model.positions)

with open("data/before_minimize.pdb","w") as f:
    app.PDBFile.writeFile(
        simulation.topology,
        simulation.context.getState(getPositions=True,enforcePeriodicBox=False).getPositions(),
        file=f,
        keepIds=True
    )

# xml outputs
with open("result/system.xml",mode="w") as f:
    f.write(XmlSerializer.serialize(system))
with open("result/integrator.xml",mode="w") as f:
    f.write(XmlSerializer.serialize(integrator))

# energy minimization
print("energy minimization start")
simulation.minimizeEnergy()
with open("data/minimized.pdb","w") as f:
    app.PDBFile.writeFile(
        simulation.topology,
        simulation.context.getState(getPositions=True,enforcePeriodicBox=False).getPositions(),
        file=f,
        keepIds=True
    )

energy minimization start


In [12]:
dcdReporter = DCDReporter("result/trajectory.dcd",reporterStep,enforcePeriodicBox=True)
dataReporter = StateDataReporter("result/log.txt",reporterStep,totalSteps=steps,step=True,speed=True,progress=True,potentialEnergy=True,temperature=True,separator="\t")
checkpointReporter = CheckpointReporter("result/checkpoint.chk",checkpointStep)

# equilibration
ts = time.perf_counter()
print("equilibration start")
simulation.context.setVelocitiesToTemperature(temperature)
simulation.step(equilibrationSteps)

with open("data/equilibrated.pdb","w") as f:
    app.PDBFile.writeFile(
        simulation.topology,
        simulation.context.getState(getPositions=True,enforcePeriodicBox=True).getPositions(),
        file=f,
        keepIds=True
    )

# repoters
simulation.reporters.append(dcdReporter)
simulation.reporters.append(dataReporter)
simulation.reporters.append(checkpointReporter)
simulation.currentStep = 0

# simulation
print("simulation start")
simulation.step(steps)

# output
simulation.saveState("result/final_state.xml")
state = simulation.context.getState(getPositions=True, enforcePeriodicBox=True)
with open("result/final_state.cif", mode="w") as file:
    PDBxFile.writeFile(simulation.topology, state.getPositions(), file)

tg = time.perf_counter()
elapse = tg - ts
h = elapse // 3600
m = (elapse % 3600) // 60
s = elapse % 60
print(f"equib - simulation: {h} h {m} min {s} sec")

equilibration start
simulation start
equib - simulation: 5.0 h 4.0 min 16.967447048053145 sec
